In [1]:
import pandas as pd
import numpy as np
import DataLoad as data
from sklearn import linear_model

In [2]:
pd.options.mode.chained_assignment = None
sessions_df = data.load_sessions_data()
users_df = data.load_users_data()
products_df = data.load_products_data()
cleared_sessions_df = sessions_df.loc[ (~sessions_df['user_id'].isna()) & (~sessions_df['product_id'].isna())]

In [3]:
def get_buys(uid):
    drop_list = []
    tmp = cleared_sessions_df.loc[(cleared_sessions_df['user_id'] == uid) & (cleared_sessions_df['event_type_BUY_PRODUCT'] == 1)]
    i = -1
    for index, row in tmp.iterrows():
        if index in drop_list:
            continue
        i += 1
        count = 1
        sum = row['offered_discount']
        j = i
        tmp2 = tmp[(i + 1):]
        for index2, row2 in tmp2.iterrows():
            j += 1
            if row['product_id'] == row2['product_id']:
                count += 1
                sum = sum + row2['offered_discount']
                tmp.drop(tmp.index[j], inplace=True)
                j -= 1
                drop_list.append(index2)
        if count > 1:
             tmp.at[index, 'offered_discount'] = sum / count
    return tmp

def get_product_category(pid):
    tmp = products_df.loc[((products_df['product_id']).isin(pid))]
    tmp = tmp.drop(['product_name', 'price', 'product_id'], axis=1)
    return tmp


In [4]:

users = users_df['user_id']
reg_list = []
for id in users:
    reg = linear_model.LinearRegression()
    user_buys = get_buys(id)
    if len(user_buys.index) > 0:
        reg.fit(get_product_category(user_buys['product_id']), user_buys['offered_discount'])
        reg_list.append([id, reg])
    else:
        reg_list.append([id, 0])

In [5]:
discounts_for_test_product = []
for i in users.index:
    if reg_list[i][1] == 0:
        discounts_for_test_product.append(0)
    else:
        discounts_for_test_product.append(reg_list[i][1].predict(get_product_category([1311])))

Prediction test(user with single buy)

In [6]:
buy = cleared_sessions_df.loc[(cleared_sessions_df['user_id'] == 140) & (cleared_sessions_df['event_type_BUY_PRODUCT'] == 1)]


In [7]:
discount = reg_list[140-102][1].predict(get_product_category([buy['product_id']]))

Same discount for product:

In [8]:
test = False
if buy['offered_discount'].iloc[0] == discount:
    test = True

In [9]:
test

True

Not all predicted test

In [10]:
predicted = 0
not_predicted = 0
for i in users.index:
    if discounts_for_test_product[i] == 0:
        not_predicted += 1
    else:
        predicted +=1

Number of predicted

In [11]:
predicted

165

Number of not predicted

In [12]:
not_predicted

35